<a href="https://colab.research.google.com/github/hritik880/Waste-Classifier/blob/master/waste_classifier_streamlit_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U ipykernel

Requirement already up-to-date: ipykernel in /usr/local/lib/python3.6/dist-packages (5.3.4)


In [ ]:
!pip install -q streamlit

In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken 1gJocRcIpqsOAVahoJH3YcMg6c0_FYQ3NNeTwaBFw3xes1vt

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
%%writefile score.py

import tensorflow as tf 
import numpy as np 
import streamlit as st
from PIL import Image 
import requests
from io import BytesIO
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array

st.set_option('deprecation.showfileUploaderEncoding', False)

st.write("""
        # Waste Classification
        """
        )

st.write("This is a simple image classification web app to predict type of waste")

@st.cache(allow_output_mutation =True)
def load_model():
  model = tf.keras.models.load_model(r'/content/drive/My Drive/Resnet_wc/models')
  return model


with  st.spinner('Loading Model....'):
  model = load_model()

labels = {0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}

def preprocess(img):
  width, height = img.shape[0], img.shape[1]
  img = image.array_to_img(img, scale=False)
  img = img.resize((256,256))
  img = image.img_to_array(img)
  img /= 255.0
  img = np.expand_dims(img,axis=0)
  return img


file = st.file_uploader("Please upload an image file", type=["jpg", "png"])

if file is None:
  st.text("You haven't uploaded an image file")
else:
  original_img = Image.open(file)
  st.image(original_img, use_column_width=True)
  img = image.img_to_array(original_img)
  a = preprocess(img)
  probs = model.predict(a)
  pred = labels[np.argmax(probs)]
  st.write(pred)
  st.text("Probability (0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash')")
  st.write(probs)


Overwriting score.py


In [ ]:
!nohup streamlit run score.py &

nohup: appending output to 'nohup.out'


In [ ]:
from pyngrok import ngrok
url = ngrok.connect(port=8501)
url

'http://656e62bd5b9f.ngrok.io'

In [ ]:
ngrok.kill()